In [8]:
from scipy.stats import spearmanr, rankdata
from scipy.spatial.distance import pdist, squareform

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import shutil
import time
import _pickle as cp
import sys
from astropy.stats import median_absolute_deviation
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MinMaxScaler
import random
from sklearn.model_selection import train_test_split


from PIL import Image

In [9]:
# xem số lượng ảnh

image_data_dir= './datasets/CIC-DDoS2019/image data'

allimgs=[]
for subdir in os.listdir(image_data_dir):
    for filename in os.listdir(os.path.join(image_data_dir,subdir)):
        filepath=os.path.join(image_data_dir,subdir,filename)
        allimgs.append(filepath)
len(allimgs) # In tổng số hình ảnh

110587

In [10]:
# Hàm phân chia các ảnh thành 3 danh sách theo tỷ lệ 60%-20%-20%
def split_images(image_paths):
    random.shuffle(image_paths)  # Xáo trộn các ảnh để phân phối ngẫu nhiên
    total_images = len(image_paths)
    
    # Tính toán số lượng ảnh cho mỗi phần
    split_1_count = int(total_images * 0.6)
    split_2_count = int(total_images * 0.2)
    
    split_1 = image_paths[:split_1_count]
    split_2 = image_paths[split_1_count:split_1_count + split_2_count]
    split_3 = image_paths[split_1_count + split_2_count:]
    
    return split_1, split_2, split_3

In [11]:
# Hàm lấy tất cả các ảnh từ thư mục con
def get_images_from_subdir(subdir):
    image_paths = []
    for dirpath, _, filenames in os.walk(subdir):
        for filename in filenames:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')):
                image_paths.append(os.path.join(dirpath, filename))
    return image_paths

In [12]:
# Hàm sao chép ảnh vào thư mục đích tương ứng
def copy_images_to_split_folders(image_list, dest_folder, subdir):
    for image_path in image_list:
        # Tạo cấu trúc thư mục con giống như trong subdir
        relative_path = os.path.relpath(image_path, subdir)
        dest_image_path = os.path.join(dest_folder, relative_path)
        
        os.makedirs(os.path.dirname(dest_image_path), exist_ok=True)  # Tạo thư mục nếu chưa tồn tại
        shutil.copy2(image_path, dest_image_path)  # Sao chép ảnh vào thư mục đích

In [13]:
# Hàm chính để chia ảnh từ từng thư mục con trong A
def split_images_in_subdirectories(root_dir, dest_dir_60, dest_dir_20_1, dest_dir_20_2):
    # Lấy danh sách các thư mục con trong thư mục A
    subdirectories = [os.path.join(root_dir, d) for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
    
    # Lặp qua từng thư mục con (B, C, D,...)
    for subdir in subdirectories:
        print(f"Đang xử lý thư mục: {subdir}")
        
        # Lấy tất cả ảnh từ thư mục con
        images = get_images_from_subdir(subdir)

        # vì dữ liệu quá nhiều, train rất lâu nên chỉ dùng 10% của tập dữ liệu
        temp_len = len(images)
        split_1_count = int(temp_len * 0.1)
        temp_of_ten_percent = images[:split_1_count]

        
        # Phân chia ảnh theo tỷ lệ 60%-20%-20%
        split_60, split_20_1, split_20_2 = split_images(temp_of_ten_percent)
        
        # Tạo thư mục con tương ứng trong các thư mục đích và sao chép ảnh
        subdir_name = os.path.basename(subdir)  # Tên thư mục con (ví dụ: B, C, D)
        copy_images_to_split_folders(split_60, os.path.join(dest_dir_60, subdir_name), subdir)
        copy_images_to_split_folders(split_20_1, os.path.join(dest_dir_20_1, subdir_name), subdir)
        copy_images_to_split_folders(split_20_2, os.path.join(dest_dir_20_2, subdir_name), subdir)

In [14]:

# Đường dẫn đến các thư mục tách ra
split_1_directory = './datasets/CIC-DDoS2019/train'
split_2_directory = './datasets/CIC-DDoS2019/val'
split_3_directory = './datasets/CIC-DDoS2019/test'
os.makedirs(split_1_directory, exist_ok=True)
os.makedirs(split_2_directory, exist_ok=True)
os.makedirs(split_3_directory, exist_ok=True)


In [15]:
# Gọi hàm chia ảnh từ các thư mục con trong A
split_images_in_subdirectories(image_data_dir, split_1_directory, split_2_directory, split_3_directory)


Đang xử lý thư mục: ./datasets/CIC-DDoS2019/image data\Benign
Đang xử lý thư mục: ./datasets/CIC-DDoS2019/image data\LDAP
Đang xử lý thư mục: ./datasets/CIC-DDoS2019/image data\MSSQL
Đang xử lý thư mục: ./datasets/CIC-DDoS2019/image data\NetBIOS
Đang xử lý thư mục: ./datasets/CIC-DDoS2019/image data\Portmap
Đang xử lý thư mục: ./datasets/CIC-DDoS2019/image data\Syn
Đang xử lý thư mục: ./datasets/CIC-DDoS2019/image data\UDP
Đang xử lý thư mục: ./datasets/CIC-DDoS2019/image data\UDPLag
